#### 本notebookは『データサイエンスの無駄遣い』（[著]篠田裕之, 翔泳社, 2021）のサンプルコードとなります。

## chapter2：多⾯的な⾃分と向き合うためのチャットボット

### 実行検証環境

・Python (3.9.2)

・pandas (1.2.4)<br>
・beautifulsoup4 (4.9.3)<br>
・urllib3 (1.26.4)<br>
・janome (0.4.1)<br>

### 事前準備

・本書の手順に応じて./dataフォルダにFacebook、LINEのチャット履歴データを保存しておく。

### パッケージのインポート

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import os

In [2]:
from janome.tokenizer import Tokenizer

In [3]:
from collections import deque
import random

### データ取得・前処理

chapter1のコードと同様にLINEとfacebookのデータを取得・前処理（ただし統計量の計算は不要のためコメントアウト）

In [7]:
def make_line_message_df(name, my_name, text):
    f = open(text, 'r', encoding='utf-8')
    line_data = f.readlines()
    f.close()    
    
    tmp_day = ""
    tmp_time = ""
    tmp_from = ""
    tmp_text = ""

    daytime_list = []
    from_list = []
    text_list = []

    count = 0
    
    for text in line_data:        
        if ("とのトーク履歴" in text) or ("保存日時" in text):
            continue

        if len(text) == 14:
            if (text[4] == "/") and (text[7] == "/") and (text[10] == "("):
                #もしメッセージが格納されていれば前の時刻までのメッセージを格納
                if len(tmp_text) > 1:
                    daytime_list.append(tmp_day + " " + tmp_time)
                    from_list.append(tmp_from)                
                    tmp_text = tmp_text.replace("\n","")                
                    text_list.append(tmp_text)
                    
                tmp_text = ""                    
                tmp_day = text[:-4]    
                continue

        if len(text) > 5:
            if text[2] == ":" and text[5] == '\t':
                #もしメッセージが格納されていれば前の時刻までのメッセージを格納
                if len(tmp_text) > 1:
                    daytime_list.append(tmp_day + " " + tmp_time)
                    from_list.append(tmp_from)                
                    tmp_text = tmp_text.replace("\n","")                
                    text_list.append(tmp_text)
                
                split_text = text.split("\t")
                tmp_time = split_text[0]
                tmp_from = split_text[1]
                tmp_text = split_text[2]                
                continue
                                
        #時刻付きの行ではない（前の時刻のメッセージの続きなら前のテキストに続ける）
        tmp_text = tmp_text + text
        
    #最後のメッセージを加える
    daytime_list.append(tmp_day + " " + tmp_time)
    from_list.append(tmp_from)                
    tmp_text = tmp_text.replace("\n","")                
    text_list.append(tmp_text)
        
    tmp_df = pd.DataFrame({
                                'from':from_list, 
                                'text':text_list,
                                'time':daytime_list    
                    })
    
    tmp_df["thread"] = name
            
    #本章では不要のためコメントアウト
    #tmp_df = get_chat_convert_df(tmp_df, name, my_name)
    
    #最後の行は削除（スレッドの最後は返信がないため）
    tmp_df = tmp_df[:-1]
    
    return tmp_df

In [8]:
def make_message_df(name, my_name, url_list):
    for i in range(len(url_list)):
        #print(url_list[i])
        #print(i)
        html = open(url_list[i], 'r', encoding='utf-8')
        soup = BeautifulSoup(html, 'html.parser')

        from_classes = soup.find_all(class_="_3-96 _2pio _2lek _2lel")
        text_classes = soup.find_all(class_="_3-96 _2let")
        time_classes = soup.find_all(class_="_3-94 _2lem")    

        from_list = []
        text_list = []
        time_list = []

        for k in range(len(from_classes)):
                from_list.append(from_classes[k].text)
                text_list.append(text_classes[k].text)
                time_list.append(time_classes[k].text)

        if i == 0:
            tmp_df = pd.DataFrame({
                            'from':from_list, 
                            'text':text_list,
                            'time':time_list
                             })        
        else:
            tmp_df2 = pd.DataFrame({
                            'from':from_list, 
                            'text':text_list,
                            'time':time_list
                             })        
            
            tmp_df = pd.concat([tmp_df, tmp_df2]).reset_index(drop=True)            
                        
    tmp_df["thread"] = name
            
    #本章では不要のためコメントアウト
    #tmp_df = get_chat_convert_df(tmp_df, name, my_name)
    
    #最後の行は削除（スレッドの最後は返信がないため）
    tmp_df = tmp_df[:-1]
    
    return tmp_df

#### LINEとFacebookのチャットデータの取得（./dataフォルダ内に分析したいチャット相手のログを取得しておく）

※GitHub(https://github.com/mirandora/ds_book/tree/main/2_1) に公開しているものはコード動作確認用に作成した架空のダミーデータとなります。

In [9]:
user_A_df = make_line_message_df("UserA","HiroyukiS","./data/user_a.txt") #LINEデータの場合
user_B_df = make_message_df("user_b","Hiroyuki Shinoda", ["./data/user_b.html"]) #Facebookデータの場合

In [10]:
#自分の発言ログのテキストのみを取得（自分のLINEアカウント名が"HiroyukiS"、facebookアカウント名が"Hiroyuki Shinoda"の場合）
selfchat_A = user_A_df[user_A_df["from"]=="HiroyukiS"].reset_index(drop=True).text
selfchat_B = user_B_df[user_B_df["from"]=="Hiroyuki Shinoda"].reset_index(drop=True).text

### 形態素解析

In [11]:
tokenizer = Tokenizer()

In [12]:
text = "むしろそのストーリーは君がつくりたまえ。"
for token in tokenizer.tokenize(text):
    print(token)

むしろ	副詞,一般,*,*,*,*,むしろ,ムシロ,ムシロ
その	連体詞,*,*,*,*,*,その,ソノ,ソノ
ストーリー	名詞,一般,*,*,*,*,ストーリー,ストーリー,ストーリー
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
君	名詞,代名詞,一般,*,*,*,君,キミ,キミ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
つくり	動詞,自立,*,*,五段・ラ行,連用形,つくる,ツクリ,ツクリ
たまえ	動詞,自立,*,*,五段・ワ行促音便,命令ｅ,たまう,タマエ,タマエ
。	記号,句点,*,*,*,*,。,。,。


In [13]:
for token in tokenizer.tokenize(text):
    print(token.surface)

むしろ
その
ストーリー
は
君
が
つくり
たまえ
。


In [14]:
word_list_A = []
for i in range(len(selfchat_A)):
    for token in tokenizer.tokenize(selfchat_A[i]):
        word_list_A.append(token.surface)

In [15]:
word_df_A = pd.DataFrame(word_list_A)
word_df_A.columns = ["word"]

In [16]:
word_df_A.word.value_counts().reset_index().head(5)

,index,word
0,。,21
1,、,14
2,？,10
3,ゲーム,10
4,て,9


In [17]:
word_list_A = []
for i in range(len(selfchat_A)):
    for token in tokenizer.tokenize(selfchat_A[i]):
        if (token.part_of_speech.split(',')[0] != '記号') and (token.part_of_speech.split(',')[0] != '助詞') and (token.part_of_speech.split(',')[0] != '助動詞'):
            word_list_A.append(token.surface)

word_df_A = pd.DataFrame(word_list_A)
word_df_A.columns = ["word"]

In [18]:
word_df_A.word.value_counts().reset_index().head(5)

,index,word
0,ゲーム,10
1,し,8
2,ほんと,5
3,いる,4
4,いや,4


In [16]:
word_df_A = word_df_A[(word_df_A["word"] != ":") & (word_df_A["word"] != "00")].reset_index(drop=True)

In [17]:
word_df_A.word.value_counts().reset_index().head(5)

,index,word
0,ゲーム,10
1,し,8
2,ほんと,5
3,いや,4
4,いる,4


### テキストデータのマルコフ連鎖

ストップワードを定義する。

In [18]:
stop_words = ["。", "！", "!", " ", "？", "?", ")"]

In [19]:
def make_model(text_list, n_size = 1):
    model = {}
    for text in text_list:
        queue = deque([], n_size)
        queue.append("[BOS]")
        
        for i, token in enumerate(tokenizer.tokenize(text)):
            key = tuple(queue)
            
            if key not in model:
                model[key] = []
                
            model[key].append(token.surface)            
            queue.append(token.surface)
            
            #ストップワードあるいは最後のtokenだった場合
            if (token.surface in stop_words) or (i == (len(list(tokenizer.tokenize(text)))-1)):                
                key = tuple(queue)

                if key not in model:
                    model[key] = []
                model[key].append("[EOS]")
                
                #もし最後のtokenではない場合はqueueをリセットして続行。
                if (i != (len(list(tokenizer.tokenize(text)))-1)):
                    queue = deque([], n_size)
                    queue.append("[BOS]")                
    return model

In [20]:
self_model_A = make_model(selfchat_A)

In [21]:
def make_sentence(model, max_sentence_num=3, seed="[BOS]", n_size=1):    
    sentence_count = 0
    c_token_list = []
    key_candidates = []

    #もしseedがBOSではない場合、まずは形態素解析。
    if seed != "[BOS]":
        #記号、助詞、助動詞いずれでもなければ文章生成元token候補リストに入れる。
        for token in tokenizer.tokenize(seed):
            if (token.part_of_speech.split(',')[0] != '記号') and (token.part_of_speech.split(',')[0] != '助詞') and (token.part_of_speech.split(',')[0] != '助動詞'):    
                c_token_list.append(token.surface)
                
        while(len(c_token_list) > 0):
            c_rand = random.randrange(len(c_token_list))        
            key_candidates = [key for key in model if key[0] == c_token_list[c_rand]] 
            if len(key_candidates) > 0:
                break
            c_token_list.pop(c_rand)        
    
    else:
        #seedから始まるkeyを取得
        key_candidates = [key for key in model if key[0] == seed]    
    
    #もし単語が辞書に存在していなかったらseedを[BOS]にしてランダムに選択
    if not key_candidates:
        key_candidates = [key for key in model if key[0] == "[BOS]"]                    
    
    #ランダムにkey選択
    m_key = random.choice(key_candidates)
    #選択したkeyからqueue生成
    queue = deque(list(m_key), n_size)

    sentence = ""
    if m_key[0] != "[BOS]":
        sentence = "".join(m_key)
    
    while(True):
        m_key = tuple(queue)        
        
        if m_key not in model:
            key_candidates = [key for key in model if key[0] == "[BOS]"]                    
            m_key = random.choice(key_candidates)
                
        next_word = random.choice(model[m_key])
        
        if next_word == "[EOS]":
            sentence_count += 1
            
            #最大文章数以上なら終了
            if sentence_count > max_sentence_num:
                break
            #最大文章数にみたない場合もランダムで終了
            if random.random() < 0.5:
                break            
                
            key_candidates = [key for key in model if key[0] == "[BOS]"]    
            m_key = random.choice(key_candidates)
            queue = deque(list(m_key), n_size)                            
            
        else:
            sentence += next_word
            queue.append(next_word)            
            
    return sentence

※架空のダミーデータでの実行のため、マルコフ連鎖で生成される文章はバリエーションが少ない点に留意。

In [31]:
make_sentence(self_model_A)

'ゲーム順調？'

In [44]:
make_sentence(self_model_A, seed="今日ご飯食べる？")

'ちょっときついな。ちょっときついな。相変わらずゲームばかりします！これほんと、これ、みんなクリアできるんだろうか？'

In [45]:
make_sentence(self_model_A, seed="今日ご飯食べる？")

'最近ゲームばかりした時間でお送りしたね！何やって聞かせている。いやほんと、どうかしてる？ゲームばかりしてリストつくるので少々お待ちを。'

In [46]:
make_sentence(self_model_A, seed="今日ご飯食べる？")

'すでに実況ゲームもやりたい、今はいかない。'